In [ ]:
import cohere
import pinecone
import pandas as pd


pinecone.init(api_key="your pinecone api key", environment="your environment")

pinecone_index_name = "semantic-search-openai"

if pinecone_index_name in pinecone.list_indexes():
    print(f"Index {pinecone_index_name} already exists. Skipping creation.")
else:
    # Load the dataset into a Pandas DataFrame
    df = pd.read_csv("wiki.csv")

    # Create an empty Pinecone index with the desired dimension
    pinecone.create_index(name=pinecone_index_name, dimension=329)

    # Populate the index with vectors for each row in the dataset
    for i, row in df.iterrows():
        text_data = row['text']
        vector = cohere_embed.embed(text_data)
        pinecone.upsert(pinecone_index_name, [vector], [i])

    print(f"Index {pinecone_index_name} created and populated successfully.")

#pinecone.deinit()

Index semantic-search-openai already exists. Skipping creation.


In [ ]:
#!pip install cohere
#!pip install --upgrade pinecone-client
#!pip install openai

#!pip install cohere_embed_api
import pinecone
import openai
import cohere
import pandas as pd
#import cohere_embed_api as co

# Initialize Pinecone and OpenAI
#pinecone.init(api_key="pinecone api key")
openai.api_key = "openai api key"

# Load the "wiki.csv" dataset into a Pandas DataFrame
df = pd.read_csv("wiki.csv")

# Define the name of the Pinecone index that contains the text data vectors
pinecone_index_name = "wiki_index"

# Define the maximum number of results to return from Pinecone and OpenAI
max_results = 5

# Prompt the user for input
user_input = str(input("Enter a question: "))

# Use the Cohere Embed API to detect the language of the input
co = cohere.Client('cohere api key')
language = co.detect_language(texts=[user_input])

# Use the Pinecone API to search for the most similar vectors in the index
if language == "unknown":
    print("Language detection failed. Please try again with a different question.")
else:
    #embedding = cohere_client.embed(texts=["Hello World","2023 will be my year"])
    vector = co.embed(texts=[user_input])
    print(vector)
    #search_results = pinecone.query(index_name=pinecone_index_name, queries=[vector], top_k=10)

    # Use the OpenAI Completion API to generate a response for the input
    completions = openai.Completion.create(
        engine="davinci",
        prompt=user_input,
        max_tokens=512,
        n=5,
        stop=None,
        temperature=0.7,
    )
    response = completions.choices[0].text.strip()

    # Print the search results and the response
    print(f"Top {max_results} search results:")
    #for i, (idx, score) in enumerate(search_results[0]):
        #text_data = df.loc[int(idx)]['text']
        #print(f"{i+1}. {text_data} (score: {score:.3f})")
    print(f"\nResponse: {response}")
    print(type(response))
    
# Deinitialize Pinecone
#pinecone.deinit()

In [ ]:
import pinecone
#print(response)
# Extract the generated text from OpenAI response
generated_text = response

# Define the Pinecone index name
#pinecone_index_name = "my-index"
pinecone.init(api_key="pinecone api key", environment="your environment")

pinecone_index_name = "semantic-search-openai"
# Insert the generated text into the Pinecone index
pinecone.upsert(index_name=pinecone_index_name, ids=["id-1"], vectors=[generated_text])

# Define the Pinecone query vector
query_vector = generated_text

# Perform a Pinecone search on the index and retrieve the top result
search_results = pinecone.query(index_name=pinecone_index_name, queries=[query_vector], top_k=1)
top_result_id = search_results[0]["id"]

# Retrieve the summary from Pinecone using the top result ID
summary = pinecone.retrieve(index_name=pinecone_index_name, ids=[top_result_id])[0]["vectors"][0]

print("Generated text:", generated_text)
print("Summary:", summary)
